In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import json
import math

In [14]:
def LSJZ(fundCode,pageIndex):

    url = 'http://api.fund.eastmoney.com/f10/lsjz'
    params = {
        'callback': 'jQuery18307633215694564663_1548321266367',
        'fundCode': fundCode,
        'pageIndex': pageIndex,
        'pageSize': 20,
    }
    cookie = 'EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; \
              EMFUND7=null; EMFUND8=null; EMFUND0=null; EMFUND9=01-24 17:11:50@#$%u957F%u4FE1%u5229%u5E7F%u6DF7%u5408A@%23%24519961; \
              st_pvi=27838598767214; st_si=11887649835514'
    headers = {
        'Cookie': cookie,
        'Host': 'api.fund.eastmoney.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
        'Referer': 'http://fundf10.eastmoney.com/jjjz_%s.html' % fundCode,
    }
    r = requests.get(url=url, headers=headers, params=params) 

    text = re.findall('\((.*?)\)', r.text)[0] 
    LSJZList = json.loads(text)['Data']['LSJZList']  
    TotalCount = json.loads(text)['TotalCount']  
    LSJZ = pd.DataFrame(LSJZList)  

    return LSJZ,TotalCount,LSJZList

In [52]:
def fund_history_data(fundCode):

    pageIndex = 1
    t_page = LSJZ(fundCode,pageIndex)[1]
    total_page = math.ceil(t_page / 20)
    pd_data = pd.concat([LSJZ(fundCode,i)[0] for i in range(total_page)])
    pd_data.index = np.arange(1,len(pd_data)+1,1)
    pd_copy = pd_data[['FSRQ','DWJZ','LJJZ','JZZZL','SGZT','SHZT']]
    pd_copy.columns = ['日期','单位净值','累计净值','日增长率','申购状态','赎回状态']
    
    def percetange(i):
        new = i+'%'
        return new
    pd_copy['日增长率'] = pd_copy['日增长率'].apply(percetange)
    
    
    address = 'C:\\Users\\lujiani\\Desktop\\东方证券\\珺容\\基金数据库\\单只基金历史数据\\%s.csv' % fundCode
    pd_copy.to_csv(address,index=False, encoding='utf_8_sig')
    return pd_copy

In [55]:
fund_history_data('270002')

C:\Users\lujiani\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,日期,单位净值,累计净值,日增长率,申购状态,赎回状态
1,2020-05-28,1.5907,4.5493,0.18%,开放申购,开放赎回
2,2020-05-27,1.5879,4.5465,-0.98%,开放申购,开放赎回
3,2020-05-26,1.6036,4.5622,0.97%,开放申购,开放赎回
4,2020-05-25,1.5882,4.5468,0.68%,开放申购,开放赎回
5,2020-05-22,1.5775,4.5361,-0.81%,开放申购,开放赎回
...,...,...,...,...,...,...
3856,2004-09-13,0.9812,0.9812,-0.45%,开放申购,封闭期
3857,2004-09-10,0.9856,0.9856,0.21%,开放申购,封闭期
3858,2004-09-09,0.9835,0.9835,-0.46%,开放申购,封闭期
3859,2004-09-08,0.9880,0.9880,-0.26%,开放申购,封闭期
